<h2 style="text-align: center;"><strong>Segment 3: Automatic Differentiation</strong></h2>

* AutoDiff with PyTorch
* AutoDiff with TensorFlow 2
* Machine Learning via Differentiation 
* Cost (or Loss) Functions
* The Future: Differentiable Programming 

**TensorFlow** and **PyTorch** are the two most popular automatic differentiation libraries.

Let's use them to calculate $dy/dx$ at $x = 5$ where: 

$$y = x^2$$

$$ \frac{dy}{dx} = 2x = 2(5) = 10 $$

In [24]:
import numpy as np
import matplotlib.pyplot as plt
import torch 
import tensorflow as tf

## **Autodiff with PyTorch**

In [25]:
import torch

In [26]:
x = torch.tensor(5.0)

In [27]:
x

tensor(5.)

In [28]:
x.requires_grad_() # contagiously track gradients through forward pass

tensor(5., requires_grad=True)

In [29]:
y = x**2

In [30]:
y.backward() # use autodiff

In [31]:
x.grad

tensor(10.)

### Autodiff with TensorFlow

In [32]:
import tensorflow as tf

In [33]:
x = tf.Variable(5.0)

In [34]:
with tf.GradientTape() as t:
    t.watch(x) # track forward pass
    y = x**2

In [35]:
t.gradient(y, x) # use autodiff

<tf.Tensor: shape=(), dtype=float32, numpy=10.0>

**Return to slides here.**

As usual, PyTorch feels more intuitive and pythonic than TensorFlow. See the standalone [*Regression in PyTorch*](https://github.com/jonkrohn/ML-foundations/blob/master/notebooks/regression-in-pytorch.ipynb) notebook for an example of autodiff paired with gradient descent in order to fit a simple regression line.